In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [7]:
%matplotlib inline


Transfer Learning for Computer Vision Tutorial
==============================================

These two major transfer learning scenarios look as follows:

-  **Finetuning the convnet**: Instead of random initialization, we
   initialize the network with a pretrained network, like the one that is
   trained on imagenet 1000 dataset. Rest of the training looks as
   usual.
-  **ConvNet as fixed feature extractor**: Here, we will freeze the weights
   for all of the network except that of the final fully connected
   layer. This last fully connected layer is replaced with a new one
   with random weights and only this layer is trained.


In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

Load Data
---------

We will use torchvision and torch.utils.data packages for loading the
data.

The problem we're going to solve today is to train a model to classify
**ants** and **bees**. We have about 120 training images each for ants and bees.
There are 75 validation images for each class. Usually, this is a very
small dataset to generalize upon, if trained from scratch. Since we
are using transfer learning, we should be able to generalize reasonably
well.

This dataset is a very small subset of imagenet.

.. Note ::
   Download the data from
   `here <https://download.pytorch.org/tutorial/hymenoptera_data.zip>`_
   and extract it to the current directory.



In [2]:
# Data augmentation and normalization for training
# Just normalization for validation
image_transforms  = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees = 45),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

dataset = 'E:\gmd_data\microsphere2_dataAugmented'
train_directory = os.path.join(dataset, 'train')
valid_directory = os.path.join(dataset, 'valid')

batch_size = 32
# num_classes = 173

data = {
    'train': datasets.ImageFolder(root=train_directory, transform=image_transforms['train']),
    'valid': datasets.ImageFolder(root=valid_directory, transform=image_transforms['valid'])

}


train_data_size = len(data['train'])
valid_data_size = len(data['valid'])

train_data = DataLoader(data['train'], batch_size=batch_size, shuffle=True)
valid_data = DataLoader(data['valid'], batch_size=batch_size, shuffle=True)

print(train_data_size, valid_data_size)

168537 42177


迁移学习
---------


这里使用ResNet-50的预训练模型。


In [10]:
resnet50 = models.resnet50(pretrained=True)

for param in resnet50.parameters():
    param.requires_grad = True

fc_inputs = resnet50.fc.in_features
resnet50.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(64, 173),
    nn.LogSoftmax(dim = 1)


)
resnet50 = resnet50.to('cuda:0')
loss_func = nn.NLLLoss()
# optimizer = optim.AdamW(resnet50.parameters(),lr=1e-4)

Training the model
------------------

Now, let's write a general function to train a model. Here, we will
illustrate:

-  Scheduling the learning rate
-  Saving the best model

In the following, parameter ``scheduler`` is an LR scheduler object from
``torch.optim.lr_scheduler``.



In [12]:
def train_and_valid(model, loss_function, epochs=25):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    history = []
    best_acc = 0.0
    best_epoch = 0

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))

        model.train()

        train_loss = 0.0
        train_acc = 0.0
        valid_loss = 0.0
        valid_acc = 0.0
#         学习率递减
        if epoch <= 10:
            LR = 1e-4;
        elif epoch <= 20:
            LR = 1e-4*0.5;
        elif epoch <= 30:
            LR = 1e-4*0.5*0.5;
        elif epoch <= 40:
            LR = 1e-4*0.5*0.5*0.5;
        else:
            LR = 1e-4*0.5*0.5*0.5*0.5;
        optimizer = optim.AdamW(resnet50.parameters(),lr=LR)

        for i, (inputs, labels) in enumerate(train_data):
            inputs = inputs.to(device)
            labels = labels.to(device)

            #因为这里梯度是累加的，所以每次记得清零
            optimizer.zero_grad()

            outputs = model(inputs)

            loss = loss_function(outputs, labels)

            loss.backward()

            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))

            acc = torch.mean(correct_counts.type(torch.FloatTensor))

            train_acc += acc.item() * inputs.size(0)

        with torch.no_grad():
            model.eval()

            for j, (inputs, labels) in enumerate(valid_data):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)

                loss = loss_function(outputs, labels)

                valid_loss += loss.item() * inputs.size(0)

                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                valid_acc += acc.item() * inputs.size(0)

        avg_train_loss = train_loss/train_data_size
        avg_train_acc = train_acc/train_data_size

        avg_valid_loss = valid_loss/valid_data_size
        avg_valid_acc = valid_acc/valid_data_size

        history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])

        if best_acc < avg_valid_acc:
            best_acc = avg_valid_acc
            best_epoch = epoch + 1

        epoch_end = time.time()

        print("Epoch: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation: Loss: {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(
            epoch+1, avg_valid_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start
        ))
        print("Best Accuracy for validation : {:.4f} at epoch {:03d}".format(best_acc, best_epoch))

        torch.save(model, dataset+'/models/'+'model_'+str(epoch+1)+'.pt')
    return model, history


In [13]:
num_epochs = 3000
trained_model, history = train_and_valid(resnet50, loss_func, num_epochs)
torch.save(history, 'models/'+dataset+'_history.pt')



Epoch: 1/3000
Epoch: 1/3000
Epoch: 001, Training: Loss: 2.7878, Accuracy: 4.7230%, 
		Validation: Loss: 2.7878, Accuracy: 15.0414%, Time: 8412.3150s
Best Accuracy for validation : 0.1504 at epoch 001
Epoch: 001, Training: Loss: 2.7878, Accuracy: 4.7230%, 
		Validation: Loss: 2.7878, Accuracy: 15.0414%, Time: 8412.3150s
Best Accuracy for validation : 0.1504 at epoch 001
Epoch: 2/3000
Epoch: 2/3000
Epoch: 002, Training: Loss: 2.2598, Accuracy: 13.3917%, 
		Validation: Loss: 2.2598, Accuracy: 18.2991%, Time: 8981.4649s
Best Accuracy for validation : 0.1830 at epoch 002
Epoch: 002, Training: Loss: 2.2598, Accuracy: 13.3917%, 
		Validation: Loss: 2.2598, Accuracy: 18.2991%, Time: 8981.4649s
Best Accuracy for validation : 0.1830 at epoch 002
Epoch: 3/3000
Epoch: 3/3000
Epoch: 003, Training: Loss: 1.6855, Accuracy: 20.6530%, 
		Validation: Loss: 1.6855, Accuracy: 30.5095%, Time: 8811.8746s
Best Accuracy for validation : 0.3051 at epoch 003
Epoch: 003, Training: Loss: 1.6855, Accuracy: 20.6530

Epoch: 23/3000
Epoch: 23/3000
Epoch: 023, Training: Loss: 0.8611, Accuracy: 69.0596%, 
		Validation: Loss: 0.8611, Accuracy: 70.8965%, Time: 8408.0875s
Best Accuracy for validation : 0.7433 at epoch 022
Epoch: 023, Training: Loss: 0.8611, Accuracy: 69.0596%, 
		Validation: Loss: 0.8611, Accuracy: 70.8965%, Time: 8408.0875s
Best Accuracy for validation : 0.7433 at epoch 022
Epoch: 24/3000
Epoch: 24/3000
Epoch: 024, Training: Loss: 0.8013, Accuracy: 70.2611%, 
		Validation: Loss: 0.8013, Accuracy: 74.0901%, Time: 8365.3434s
Best Accuracy for validation : 0.7433 at epoch 022
Epoch: 024, Training: Loss: 0.8013, Accuracy: 70.2611%, 
		Validation: Loss: 0.8013, Accuracy: 74.0901%, Time: 8365.3434s
Best Accuracy for validation : 0.7433 at epoch 022
Epoch: 25/3000
Epoch: 25/3000
Epoch: 025, Training: Loss: 0.7899, Accuracy: 71.2045%, 
		Validation: Loss: 0.7899, Accuracy: 73.2058%, Time: 8423.8834s
Best Accuracy for validation : 0.7433 at epoch 022
Epoch: 025, Training: Loss: 0.7899, Accuracy:

Epoch: 45/3000
Epoch: 45/3000
Epoch: 045, Training: Loss: 0.7106, Accuracy: 84.2278%, 
		Validation: Loss: 0.7106, Accuracy: 77.7556%, Time: 9233.5630s
Best Accuracy for validation : 0.8132 at epoch 038
Epoch: 045, Training: Loss: 0.7106, Accuracy: 84.2278%, 
		Validation: Loss: 0.7106, Accuracy: 77.7556%, Time: 9233.5630s
Best Accuracy for validation : 0.8132 at epoch 038
Epoch: 46/3000
Epoch: 46/3000
Epoch: 046, Training: Loss: 0.6661, Accuracy: 84.3643%, 
		Validation: Loss: 0.6661, Accuracy: 80.0507%, Time: 9920.7980s
Best Accuracy for validation : 0.8132 at epoch 038
Epoch: 046, Training: Loss: 0.6661, Accuracy: 84.3643%, 
		Validation: Loss: 0.6661, Accuracy: 80.0507%, Time: 9920.7980s
Best Accuracy for validation : 0.8132 at epoch 038
Epoch: 47/3000
Epoch: 47/3000
Epoch: 047, Training: Loss: 0.6815, Accuracy: 84.6152%, 
		Validation: Loss: 0.6815, Accuracy: 78.2701%, Time: 12759.4504s
Best Accuracy for validation : 0.8132 at epoch 038
Epoch: 047, Training: Loss: 0.6815, Accuracy

KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
history = np.array(history)
plt.plot(history[:, 0:2])
plt.legend(['Tr Loss', 'Val Loss'])
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.ylim(0, 1)
plt.savefig(dataset+'_loss_curve.png')
plt.show()

plt.plot(history[:, 2:4])
plt.legend(['Tr Accuracy', 'Val Accuracy'])
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.savefig(dataset+'_accuracy_curve.png')
plt.show()

In [3]:
# 将验证集和真值对比图画出来 导入模型
dataset = 'E:\gmd_data\microsphere2_dataAugmented'
model38 = torch.load( dataset+'/models/'+'model_38.pt')
for param in model38.parameters():
    param.requires_grad = False
model38 = model38.to('cuda:0')
loss_function = nn.NLLLoss()



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
history = []
best_acc = 0.0
best_epoch = 0


valid_loss = 0.0
valid_acc = 0.0

for j, (inputs, labels) in enumerate(valid_data):
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model38(inputs)

    loss = loss_function(outputs, labels)
    print("valid_data_j: {}".format(j))
    print("Outputs : ")
    print(outputs)
    print("labels ：")
    print(labels)
    print("loss ：")
    print(loss)

    valid_loss += loss.item() * inputs.size(0)

    ret, predictions = torch.max(outputs.data, 1)
    correct_counts = predictions.eq(labels.data.view_as(predictions))

    acc = torch.mean(correct_counts.type(torch.FloatTensor))

    valid_acc += acc.item() * inputs.size(0)

avg_valid_loss = valid_loss/valid_data_size
avg_valid_acc = valid_acc/valid_data_size

valid_data_j: 0
Outputs : 
tensor([[-100.2883,  -92.3576,  -92.1173,  ...,  -54.4081, -108.0811,
          -98.8013],
        [ -65.5693,  -67.4664,  -63.6391,  ...,  -73.7634,  -63.7410,
          -68.3932],
        [ -64.6967,  -61.4826,  -60.9130,  ...,  -87.1294,  -50.0973,
          -56.6066],
        ...,
        [ -50.7181,  -85.8765,  -71.2357,  ..., -110.1529,  -99.7539,
          -98.7987],
        [ -98.8066,  -84.0519,  -87.3415,  ...,  -39.6289,  -74.1971,
          -78.0432],
        [ -80.4934,  -80.3928,  -90.7395,  ...,  -20.4374,  -74.3231,
          -72.3688]], device='cuda:0')
labels ：
tensor([125,  32,  90, 162, 109,  90,  24, 165,  78,  58,  49,  26, 121, 122,
         67, 158, 128,  71,  25,  89, 130, 163,   2,  21, 110,  61,  16,  81,
        138, 147,  98, 117], device='cuda:0')
loss ：
tensor(0.5662, device='cuda:0')
valid_data_j: 1
Outputs : 
tensor([[ -56.7027,  -75.4782,  -71.4482,  ...,  -64.2741,  -74.9183,
          -76.6378],
        [ -88.4017,  -77.324

In [4]:
history.append([avg_valid_loss, avg_valid_acc])

print("Validation: Loss: {:.4f}, Accuracy: {:.4f}%".format(
    avg_valid_loss, avg_valid_acc*100
))

Validation: Loss: 0.6523, Accuracy: 81.3216%


In [5]:
torch.max(outputs)

tensor(-0.4405, device='cuda:0')